# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
Arxiv has 74 new papers today
          1 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/1 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2403.03125


extracting tarball to tmp_2403.03125...

 done.


E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']


Found 118 bibliographic references in tmp_2403.03125/main.bbl.
syntax error in line 417: '=' expected


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2403.03125-b31b1b.svg)](https://arxiv.org/abs/arXiv:2403.03125) | **A3COSMOS & A3GOODSS: Continuum Source Catalogues and Multi-band Number  Counts**  |
|| S. Adscheid, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-03-06*|
|*Comments*| *27 pages, 14 figures, A&A in press. Catalogues will be released at the CDS, see this https URL , and on the A3COSMOS website, see this https URL*|
|**Abstract**| Galaxy submillimetre number counts are a fundamental measurement in our understanding of galaxy evolution models. Most early measurements are obtained via single-dish telescopes with substantial source confusion, whereas recent interferometric observations are limited to small areas. We used a large database of ALMA continuum observations to accurately measure galaxy number counts in multiple (sub)millimetre bands, thus bridging the flux density range between single-dish surveys and deep interferometric studies. We continued the Automated Mining of the ALMA Archive in the COSMOS Field project (A3COSMOS) and extended it with observations from the GOODS-South field (A3GOODSS). The database consists of ~4,000 pipeline-processed continuum images from the public ALMA archive, yielding 2,050 unique detected sources. To infer galaxy number counts, we constructed a method to reduce the observational bias inherent to targeted pointings that dominate the database. This method comprises a combination of image selection, masking, and source weighting. The effective area was calculated by accounting for inhomogeneous wavelengths, sensitivities, and resolutions and for spatial overlap between images. We tested and calibrated our method with simulations. We obtained the first number counts derived in a consistent and homogeneous way in four different ALMA bands covering a relatively large area. The results are consistent with number counts from the literature within the uncertainties. We extended the available depth in ALMA Band 4 by 0.4 dex with respect to previous studies. In Band 7, at the depth of the inferred number counts, ~40% of the cosmic infrared background is resolved into discrete sources. This fraction, however, decreases with wavelength, reaching ~4% in Band 3. Finally, we used the number counts to test models of dusty galaxy evolution, and find a good agreement within the uncertainties. |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2403.03125.md
    + _build/html/tmp_2403.03125/./a3_coverage_all_bands.png
    + _build/html/tmp_2403.03125/./NC_final.png
    + _build/html/tmp_2403.03125/./Eff_area_database_vs_NC.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\acosmos}[0]{A^3COSMOS\xspace}$
$\newcommand{\agoodss}[0]{A^3GOODSS\xspace}$</div>



<div id="title">

# $\acosmos$ \& $\agoodss$: Continuum source catalogues and multi-band number counts

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2403.03125-b31b1b.svg)](https://arxiv.org/abs/2403.03125)<mark>Appeared on: 2024-03-06</mark> -  _27 pages, 14 figures, A&A in press. Catalogues will be released at the CDS, see this https URL , and on the A3COSMOS website, see this https URL_

</div>
<div id="authors">

S. Adscheid, et al. -- incl., <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Galaxy submillimetre number counts are a fundamental measurement in our understanding of galaxy evolution models.   Most early measurements are obtained via single-dish telescopes with substantial source confusion, whereas recent interferometric observations are limited to small areas. We used a large database of ALMA continuum observations to accurately measure galaxy number counts in multiple (sub)millimetre bands, thus bridging the flux density range between single-dish surveys and deep interferometric studies. We continued the Automated Mining of the ALMA Archive in the COSMOS Field project ( $\acosmos$ ) and extended it with observations from the GOODS-South field ( $\agoodss$ ).   The database consists of $\sim$ 4,000 pipeline-processed continuum images from the public ALMA archive, yielding 2,050 unique detected sources,   including sources   with and without a known optical counterpart.   To infer galaxy number counts, we constructed a method to reduce the observational bias inherent to targeted pointings that dominate the database.   This method comprises a combination of image selection, masking, and source weighting.   The effective area was calculated by accounting for inhomogeneous wavelengths, sensitivities, and resolutions and for the spatial overlap between images.   We tested and calibrated our method with simulations. We obtained the first number counts derived in a consistent and homogeneous way in four different ALMA bands covering a relatively large area.   The results are consistent with number counts retrieved from the literature within the uncertainties.   We extended the available depth in ALMA Band 4 by 0.4 dex with respect to previous studies.   In Band 7, at the depth of the inferred number counts, $\sim$ 40 \% of the cosmic infrared background is resolved into discrete sources. This fraction, however, decreases with increasing wavelength, reaching $\sim$ 4 \% in Band 3.   Finally, we used the number counts to test models of dusty galaxy evolution, and find a good agreement within the uncertainties. By continuing the $\acosmos$ and $\agoodss$ archival effort, we obtained the deepest archive-based (sub)millimetre number counts  measured to date over such a wide area. This database proves to be a valuable resource that, thanks to its substantial size, can be used for statistical analyses after having applied certain conservative restrictions.

</div>

<div id="div_fig1">

<img src="tmp_2403.03125/./a3_coverage_all_bands.png" alt="Fig1" width="100%"/>

**Figure 1. -** Spatial sky coverage of all successfully imaged ALMA archival data available for COSMOS (_top panel_) and GOODS-S (_bottom panel_) as of June 6, 2022. The blue shaded shapes are outlines of single pointings and blind surveys; the darker shades of blue indicate an overlap of multiple images. For single pointings, the circle size corresponds to the area where the primary beam attenuation is >0.5. The outlines of blind surveys have been traced manually. For easier visibility the outlines of surveys are only drawn once, even if multiple coverages of the same survey exist in our database. The dashed coloured lines indicate the approximate outlines of different ancillary survey fields. In COSMOS: CANDELS \citep[yellow;][]{grogin11}, COSMOS _HST_/ACS imaging \citep[orange;][]{koekemoer07}, and S2COSMOS \citep[red;][]{simpson19}. In GOODS-S: Hubble Ultra Deep Field \citep[yellow;][]{hudf}, CANDELS \citep[orange; as in][]{guo13}, and Hubble Legacy Fields \citep[red;][]{hlf}. (*sky_coverage*)

</div>
<div id="div_fig2">

<img src="tmp_2403.03125/./NC_final.png" alt="Fig10" width="100%"/>

**Figure 10. -** Number counts inferred from the $\acosmos$ database in four ALMA bands: red, Band 3 (2.6--3.6 mm); orange, Band 4 (1.8--2.4 mm); green, Band 6 (1.1--1.4 mm); blue, Band 7 (0.8--1.1 mm).
  Number counts were measured by combining blind surveys and single pointings containing a detected source within 1 arcsec radius around the phase centre and excluding a central area of 1 arcsec radius in Band 3, 4, and 6, and 4 arcsec in Band 7. _Top panel_: Differential. _Bottom panel_: Cumulative. The number counts calculated in this work are shown as solid lines, with the shaded area indicating the uncertainty resulting from flux density, contamination, and completeness errors, quadratically combined with Poisson errors.
  The respective flux densities of the individual sources are shown as points in the cumulative panel.
  The vertical dotted lines indicate the flux density above which our number counts can be considered  complete.
  Number counts from the literature are shown as symbols and are colour-coded to the respective ALMA band. (*figure-nc-all*)

</div>
<div id="div_fig3">

<img src="tmp_2403.03125/./Eff_area_database_vs_NC.png" alt="Fig2" width="100%"/>

**Figure 2. -** Cumulative areal coverage of the $\acosmos$ database, version \texttt{20220606}, as a function of 1 $\sigma_{\rm RMS}$ sensitivity for ALMA Bands 3, 4, 6, and 7. The noise of all images in each band is normalised to the central wavelength of the band, and spatial overlaps of images are accounted for. The solid lines indicate an  entire database; the image areas are considered out to a primary beam attenuation of 0.2. The  dashed lines are images selected for the computation of the number counts after masking out the phase centres and cropping the edges at a primary beam attenuation of 0.3 (see Sect. \ref{subsec_selection_criteria}). The symbols (see legend) represent the area and normalised depth of multiple ALMA blind surveys: ASPECS-LP \citep{gonzales-lopez19, gonzales-lopez20}, MORA \citep{zavala21,casey21}, ASAGAO \citep{hatsukade18}, and GOODS-ALMA \citep{gomez-guijarro21}.  (*sensitivity_curve*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2403.03125"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

378  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

3  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
